In [1]:
import matplotlib
#import tensorflow as tf
matplotlib.use('Agg')
#import matplotlib.pyplot as plt
#from matplotlib import rc
import pandas as pd

from keras.layers import Input
from keras.layers.core import Activation, Dense, Dropout, Permute
from keras.layers import Embedding
from keras.layers import add, concatenate, dot, multiply
from keras.layers import LSTM,GRU
from keras.models import Model
#from keras.preprocessing.sequences import pad_sequences
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
import collections
import itertools
import nltk
import numpy as np
import matplotlib.pyplot as plt
import os
from keras.utils.vis_utils import plot_model
import keras.backend as K
import nltk
#import tensorflow as tf
#import time
import tensorflow as tf
from tensorflow.keras.utils import pad_sequences

import json
import pandas as pd

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
# Opening JSON file
f = open("/content/drive/MyDrive/IIITH/CVProject/dataset.json")
# converting csv file to data frame
data_frame = pd.read_csv("/content/drive/MyDrive/IIITH/CVProject/KGfacts-CloseWorld.csv",
                         names=['Qid', 'fact', 'factvalue', 'col4', 'col5'])
data = json.load(f)


In [4]:
data_frame.count()

Qid          174008
fact         174007
factvalue    173807
col4            680
col5             13
dtype: int64

In [5]:
def createType1Story(Qid, imageid, name, pos, total):
    return str(imageid) + '|' + Qid + '|' + name + ' is at position ' + str(pos) + ' of ' + str(total)

def createType2Story(imageid, wikiCap):
    return imageid + '|' + '|'  + wikiCap

def createType3Story(Qid, imageid, name1 , data_frame ):
    df = data_frame.loc[data_frame['Qid'] == Qid]
    type3stories = []
    for ind in df.index:
        type3stories.append(str(imageid) + '|' + Qid + '|' + str(name1)  + " "  + df['fact'][ind]  + " " + str(df['factvalue'][ind]) ) # \
        #+ " " + df['col4'][ind]) # + " " + df['col5'][ind]
    return type3stories

def createType1QA(imageid, question, answer):
    return str(imageid) + '| |' + question + '|' + str(answer)



In [6]:
def createStoriesQA(imageid, data):
    Qids = data[imageid]['Qids']
    questions_list = data[imageid]['Questions']
    answers_list = data[imageid]['Answers']
    wikiCap = data[imageid]['wikiCap']
    NamedEntities = data[imageid]['NamedEntities']
    paraQuestions_list = data[imageid]['ParaQuestions']
    storyitems = []

    # Create story items / knowledge base items
    for Qid, name in zip(Qids, NamedEntities):
        pos = Qids.index(Qid) + 1
        total = len(Qids)
        storyitems1 = createType1Story(Qid, imageid, name, pos, total)
        storyitems.append(storyitems1)
        storyitems2 = createType2Story(imageid, wikiCap)
        storyitems.append(storyitems2)
        storyitems3 = createType3Story(Qid, imageid, name, data_frame)
        storyitems.extend(storyitems3)

    qaitems = []
    for question, answer in zip(questions_list, answers_list):
        qaitem = createType1QA(imageid, question, answer)
        qaitems.append(qaitem)

#    print(len(qaitems))
    return storyitems, qaitems

In [7]:
# returns JSON object as
# a dictionary
#data = json.load(f)
imageid_list=[]
# Iterating through the json
count = 0
storyitems_list=[]
qaitems_list = []
for i in data:
    # REMOVE THIS TO RUN FOR FULL VOLUME
    # REMOVE THIS TO RUN FOR FULL VOLUME
    # REMOVE THIS TO RUN FOR FULL VOLUME
    count = count + 1
    if (count > 1000):
        break
    # REMOVE THIS TO RUN FOR FULL VOLUME
    # REMOVE THIS TO RUN FOR FULL VOLUME
    # REMOVE THIS TO RUN FOR FULL VOLUME
    imageid_list.append(i)
    storyitems, qaitems = createStoriesQA(i, data)
    storyitems_list.append(storyitems)
    qaitems_list.append(qaitems)
print(storyitems_list[0])
print(qaitems_list[0])
print(len(storyitems_list))
print(len(qaitems_list))

['21717|Q1441444|Francis Condon is at position 1 of 1', '21717||Francis Condon in the early 20th Century', '21717|Q1441444|Francis Condon alma mater Georgetown University Law Center', '21717|Q1441444|Francis Condon date of birth 1891-11-11', '21717|Q1441444|Francis Condon date of death 23-11-1965', '21717|Q1441444|Francis Condon is member of Democratic Party', '21717|Q1441444|Francis Condon knows language English', '21717|Q1441444|Francis Condon occupation judge', '21717|Q1441444|Francis Condon occupation lawyer', '21717|Q1441444|Francis Condon occupation politician', '21717|Q1441444|Francis Condon place of birth Central Falls', '21717|Q1441444|Francis Condon place of death Boston', '21717|Q1441444|Francis Condon sex male']
['21717| |Who is the person in the image?|Francis Condon', '21717| |For how many years did the person in the image live?|74', '21717| |Was the person in the image born after the end of World War II?|No', '21717| |In which country was the person in the image born?|Un

In [8]:
def get_data_new(storyitems_list, qaitems_list):
    imageids ,stories, questions, answers = [],[],[],[]

    #get the stories for each image
    for storyitems in storyitems_list:
      storylst = []
      for story in storyitems:
        story_list = story.split("|")
        #print(story_list)
        story_text = story_list[2]
        #print('story text' , story_text)
        imageid = story_list[0]
        storylst.append(story_text)
      imageids.append(imageid)
      stories.append(storylst)

    for qaitems in qaitems_list:
      questionlst = []
      answerlst = []
      for qa in qaitems:
        qalist = qa.split("|")
        question = qalist[2]
        answer = qalist[3]
        imageid = qalist[0]
        questionlst.append(question)
        answerlst.append(answer)
      questions.append(questionlst)
      answers.append(answerlst)

    return imageids, stories, questions, answers


# PREPARE DATA FOR TRAINING THE MODEL

In [9]:
DATA_DIR = './'
MODEL_DIR='./'

In [10]:
def listToString(s):
 
    # initialize an empty string
    str1 = ""
 
    # traverse in the string
    for ele in s:
        str1 += ele + ". " 
 
    # return string
    return str1 


In [11]:
imageids, stories, questions, answers = get_data_new(storyitems_list, qaitems_list)
print(imageids[0])
print(listToString(stories[0]))
print(questions[0])
print(answers[0])


21717
Francis Condon is at position 1 of 1. Francis Condon in the early 20th Century. Francis Condon alma mater Georgetown University Law Center. Francis Condon date of birth 1891-11-11. Francis Condon date of death 23-11-1965. Francis Condon is member of Democratic Party. Francis Condon knows language English. Francis Condon occupation judge. Francis Condon occupation lawyer. Francis Condon occupation politician. Francis Condon place of birth Central Falls. Francis Condon place of death Boston. Francis Condon sex male. 
['Who is the person in the image?', 'For how many years did the person in the image live?', 'Was the person in the image born after the end of World War II?', 'In which country was the person in the image born?', 'In which continent was the person in the image born?', 'How many political parties has the person in the image been a part of?', 'Who is the founder of the political party to which person in the image belongs to?', 'Is the person in the image a judge?']
['Fra

In [12]:
# GENERATE THE TRAIN JSON 
data = []
for story_list, question_list, answer_list in zip(stories[:800], questions[:800], answers[:800]):
  qas = []
  context = listToString(story_list)

  for i, question in enumerate(question_list):
      answer = answer_list[i]
      qas.append({
          "id": f"{i+1:05d}",
          "is_impossible": False if answer else True,
          "question": question,
          "answers": [{"text": answer, "answer_start": context.find(answer)}] if answer else []
      })

  data.append({
      "context": context,
      "qas": qas
  })

json_data_train = json.dumps(data)
print(json_data_train)


[{"context": "Francis Condon is at position 1 of 1. Francis Condon in the early 20th Century. Francis Condon alma mater Georgetown University Law Center. Francis Condon date of birth 1891-11-11. Francis Condon date of death 23-11-1965. Francis Condon is member of Democratic Party. Francis Condon knows language English. Francis Condon occupation judge. Francis Condon occupation lawyer. Francis Condon occupation politician. Francis Condon place of birth Central Falls. Francis Condon place of death Boston. Francis Condon sex male. ", "qas": [{"id": "00001", "is_impossible": false, "question": "Who is the person in the image?", "answers": [{"text": "Francis Condon", "answer_start": 0}]}, {"id": "00002", "is_impossible": false, "question": "For how many years did the person in the image live?", "answers": [{"text": "74", "answer_start": -1}]}, {"id": "00003", "is_impossible": false, "question": "Was the person in the image born after the end of World War II?", "answers": [{"text": "No", "an

In [13]:
# GENERATE THE TEST JSON 
data = []
for story_list, question_list, answer_list in zip(stories[800:], questions[800:], answers[800:]):
  qas = []
  context = listToString(story_list)

  for i, question in enumerate(question_list):
      answer = answer_list[i]
      qas.append({
          "id": f"{i+1:05d}",
          "is_impossible": False if answer else True,
          "question": question,
          "answers": [{"text": answer, "answer_start": context.find(answer)}] if answer else []
      })

  data.append({
      "context": context,
      "qas": qas
  })

json_data_test = json.dumps(data)
print(json_data_test)

[{"context": "George W. Bush is at position 1 of 2. Ferenc Somogyi with President Bush upon presentation of his credentials as Ambassador to the U.S., September 18, 2007. George W. Bush alma mater Harvard Business School. George W. Bush alma mater Harvard University. George W. Bush alma mater Phillips Academy. George W. Bush alma mater The Kinkaid School. George W. Bush alma mater Yale College. George W. Bush date of birth 06-07-1946. George W. Bush is member of Republican Party. George W. Bush knows language American English. George W. Bush knows language English. George W. Bush knows language Spanish. George W. Bush native language English. George W. Bush occupation politician. George W. Bush occupation statesperson. George W. Bush place of birth New Haven. George W. Bush religion United Methodist Church. George W. Bush sex male. George W. Bush spouse Laura Bush. Ferenc Somogyi is at position 2 of 2. Ferenc Somogyi with President Bush upon presentation of his credentials as Ambassado

In [14]:
!pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 98.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 91.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 76.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 80.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.2/224.2 kB 26.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.8/198.8 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [15]:
import logging

from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

In [16]:
model_type="bert"
model_name= "bert-base-cased"
if model_type == "bert":
    model_name = "bert-base-cased"

elif model_type == "roberta":
    model_name = "roberta-base"

elif model_type == "distilbert":
    model_name = "distilbert-base-cased"

elif model_type == "distilroberta":
    model_type = "roberta"
    model_name = "distilroberta-base"

elif model_type == "electra-base":
    model_type = "electra"
    model_name = "google/electra-base-discriminator"

elif model_type == "electra-small":
    model_type = "electra"
    model_name = "google/electra-small-discriminator"

elif model_type == "xlnet":
    model_name = "xlnet-base-cased"

In [17]:
# Configure the model 
model_args = QuestionAnsweringArgs()
model_args.train_batch_size = 16
model_args.evaluate_during_training = True
model_args.n_best_size=3
model_args.num_train_epochs=5


In [18]:
### Advanced Methodology
train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": f"outputs/{model_type}",
    "best_model_dir": f"outputs/{model_type}/best_model",
    "evaluate_during_training": True,
    "max_seq_length": 128,
    "num_train_epochs": 5,
    "evaluate_during_training_steps": 1000,
    "wandb_project": "Question Answer Application",
    "wandb_kwargs": {"name": model_name},
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "n_best_size":3
    # "use_early_stopping": True,
    # "early_stopping_metric": "mcc",
    # "n_gpu": 2,
    # "manual_seed": 4,
    # "use_multiprocessing": False,
    #"train_batch_size": 128,
    #"eval_batch_size": 64,
    # "config": {
    #     "output_hidden_states": True
    # }
}

In [22]:

model = QuestionAnsweringModel(
    model_type,model_name, args=train_args,
    use_cuda=True
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForQuestionAnswering: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and a

In [23]:
### Remove output folder
!rm -rf outputs

In [24]:
# Train the model
model.train_model(json.loads(json_data_train), eval_data=json.loads(json_data_test))

convert squad examples to features:   0%|          | 0/6037 [00:00<?, ?it/s]Could not find answer: '' vs. '74'
Could not find answer: '' vs. 'No'
Could not find answer: '' vs. 'United States of America'
Could not find answer: '' vs. 'North America'
Could not find answer: '' vs. 'Andrew Jackson'
Could not find answer: '' vs. 'Yes'
Could not find answer: '' vs. 'Yes'
Could not find answer: '' vs. 'United States of America'
Could not find answer: '' vs. 'North America'
Could not find answer: '' vs. 'Abraham Lincoln'
Could not find answer: '' vs. 'No'
Could not find answer: '' vs. 'Yes'
Could not find answer: '' vs. 'People's Republic of China'
Could not find answer: '' vs. 'Asia'
Could not find answer: '' vs. 'Chen Duxiu'
Could not find answer: '' vs. 'No'
Could not find answer: '' vs. '64'
Could not find answer: '' vs. 'No'
Could not find answer: '' vs. 'United Kingdom'
Could not find answer: '' vs. 'Europe'
Could not find answer: '' vs. 'No'
Could not find answer: '' vs. 'Person in the 

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Training loss,▁
global_step,▁
lr,▁
Training loss,3.7533
global_step,50
lr,2e-05


Running Epoch 0 of 5:   0%|          | 0/276 [00:00<?, ?it/s]


convert squad examples to features: 100%|██████████| 1585/1585 [00:08<00:00, 190.65it/s]

add example index and unique id: 100%|██████████| 1585/1585 [00:00<00:00, 549919.09it/s]


...

Running Evaluation:   0%|          | 0/207 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/207 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/276 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/207 [00:00<?, ?it/s]

(1380,
 {'global_step': [276, 552, 828, 1000, 1104, 1380],
  'correct': [3, 2, 3, 3, 3, 3],
  'similar': [10, 11, 10, 10, 7, 12],
  'incorrect': [7, 7, 7, 7, 10, 5],
  'train_loss': [0.9099349975585938,
   0.7731256484985352,
   1.5239826440811157,
   0.0006103813648223877,
   0.1640845537185669,
   0.00477046612650156],
  'eval_loss': [-7.072709088164252,
   -7.706257548309178,
   -7.783099335748792,
   -8.162420742753623,
   -8.41827068236715,
   -8.460711050724637]})

In [25]:

# Evaluate the model
result, texts = model.eval_model(json.loads(json_data_train))

add example index and unique id: 100%|██████████| 6037/6037 [00:00<00:00, 605490.64it/s]


Running Evaluation:   0%|          | 0/796 [00:00<?, ?it/s]

In [38]:
# Make predictions with the model
to_predict = [
    {
        "context": "Susilo Bambang Yudhoyono is at position 1 of 2. Najib with President Susilo Bambang Yudhoyono in Putrajaya on 18 December 2012.. Susilo Bambang Yudhoyono alma mater Bogor Agricultural University. Susilo Bambang Yudhoyono alma mater Fort Benning. Susilo Bambang Yudhoyono alma mater Military Academy. Susilo Bambang Yudhoyono alma mater Ranger School. Susilo Bambang Yudhoyono alma mater Sepuluh Nopember Institute of Technology. Susilo Bambang Yudhoyono alma mater United States Army Airborne School. Susilo Bambang Yudhoyono alma mater United States Army Command and General Staff College. Susilo Bambang Yudhoyono alma mater Webster University. Susilo Bambang Yudhoyono date of birth 1949-09-09. Susilo Bambang Yudhoyono is member of Democratic Party. Susilo Bambang Yudhoyono knows language English. Susilo Bambang Yudhoyono knows language Indonesian. Susilo Bambang Yudhoyono occupation military officer. Susilo Bambang Yudhoyono occupation politician. Susilo Bambang Yudhoyono place of birth Tremas. Susilo Bambang Yudhoyono religion Islam. Susilo Bambang Yudhoyono sex male. Susilo Bambang Yudhoyono spouse Kristiani Herawati. Najib Razak is at position 2 of 2. Najib with President Susilo Bambang Yudhoyono in Putrajaya on 18 December 2012.. Najib Razak alma mater Gadjah Mada University. Najib Razak alma mater University of Nottingham. Najib Razak date of birth 1953-07-23. Najib Razak is member of United Malays National Organisation. Najib Razak occupation politician. Najib Razak place of birth Kuala Lumpur. Najib Razak religion Islam. Najib Razak sex male. Najib Razak spouse Rosmah Mansor.",
        "qas": [
            {
                "question": "Who went to Bogor Agricultural University?",
                "id": "0",
            }
        ],
    }
]
     

In [39]:
answers, probabilities = model.predict(to_predict)

print(answers)

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 384.02it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[{'id': '0', 'answer': ['', 'December']}]
